In [15]:
import polars as pl
from pathlib import Path

In [16]:
# Set the date for your data partition
DATE = "2025-07-13"  # Change as needed

In [17]:
player_path = Path(f"../data/processed/player/{DATE}/player.parquet")
battlelog_path = Path(f"../data/processed/player/{DATE}/battlelog.parquet")
club_path = Path(f"../data/processed/club/{DATE}/club.parquet")
club_members_path = Path(f"../data/processed/club/{DATE}/club_members.parquet")

In [18]:
player_df = pl.read_parquet(player_path)
battlelog_df = pl.read_parquet(battlelog_path)
club_df = pl.read_parquet(club_path)
club_members_df = pl.read_parquet(club_members_path)

In [19]:
battlelog_df.head(1)

battle_time,event_mode,event_map,battle_mode,battle_type,battle_result,battle_duration,player_tag,player_name,brawler_name,brawler_power,brawler_trophies,team_size,opponent_count,is_star_player,extracted_at
str,str,str,str,str,str,i64,str,str,str,i64,i64,i64,i64,bool,datetime[μs]
"""20250713T061819.000Z""","""unknown""","""unknown""","""hotZone""","""ranked""","""defeat""",102,"""#GQJRYV0JQ""","""TT•Samu24bs""","""MORTIS""",11,1000,3,3,false,2025-07-13 15:21:58.011851


In [20]:
battlelog_df["battle_type"].unique()

battle_type
str
"""ranked"""
"""soloRanked"""


In [21]:
winrate_df = (
    battlelog_df.group_by("player_tag")
    .agg(
        [
            pl.len().alias("total_games"),
            pl.col("battle_result").eq("victory").sum().alias("wins"),
        ]
    )
    .with_columns([(pl.col("wins") / pl.col("total_games")).alias("winrate")])
)

In [22]:
display(winrate_df)

player_tag,total_games,wins,winrate
str,u32,u32,f64
"""#9QUU8P8RG""",25,19,0.76
"""#GCCYL880J""",24,14,0.583333
"""#G02QL2U2""",9,4,0.444444
"""#RPQJ09GU""",25,17,0.68
"""#8UUQLGPVQ""",25,18,0.72
…,…,…,…
"""#PJVJG8G""",23,11,0.478261
"""#9R9LPJ8RV""",25,17,0.68
"""#CJYGRUJ9""",11,6,0.545455


In [23]:
player_df.schema

Schema([('tag', String),
        ('name', String),
        ('trophies', Int64),
        ('highest_trophies', Int64),
        ('exp_level', Int64),
        ('exp_points', Int64),
        ('three_vs_three_victories', Int64),
        ('solo_victories', Int64),
        ('duo_victories', Int64),
        ('club_name', String),
        ('club_tag', String),
        ('total_brawlers', Int64),
        ('maxed_brawlers', Int64),
        ('total_brawler_trophies', Int64),
        ('extracted_at', Datetime(time_unit='us', time_zone=None))])

In [29]:
battlelog_df.schema

Schema([('battle_time', String),
        ('event_mode', String),
        ('event_map', String),
        ('battle_mode', String),
        ('battle_type', String),
        ('battle_result', String),
        ('battle_duration', Int64),
        ('player_tag', String),
        ('player_name', String),
        ('brawler_name', String),
        ('brawler_power', Int64),
        ('brawler_trophies', Int64),
        ('team_size', Int64),
        ('opponent_count', Int64),
        ('is_star_player', Boolean),
        ('extracted_at', Datetime(time_unit='us', time_zone=None))])

In [28]:
battlelog_df.head()

battle_time,event_mode,event_map,battle_mode,battle_type,battle_result,battle_duration,player_tag,player_name,brawler_name,brawler_power,brawler_trophies,team_size,opponent_count,is_star_player,extracted_at
str,str,str,str,str,str,i64,str,str,str,i64,i64,i64,i64,bool,datetime[μs]
"""20250713T061819.000Z""","""unknown""","""unknown""","""hotZone""","""ranked""","""defeat""",102,"""#GQJRYV0JQ""","""TT•Samu24bs""","""MORTIS""",11,1000,3,3,false,2025-07-13 15:21:58.011851
"""20250712T191751.000Z""","""unknown""","""unknown""","""hotZone""","""ranked""","""victory""",82,"""#GQJRYV0JQ""","""TT•Samu24bs""","""TARA""",11,1000,3,3,false,2025-07-13 15:21:58.011883
"""20250712T191544.000Z""","""unknown""","""unknown""","""hotZone""","""ranked""","""victory""",94,"""#GQJRYV0JQ""","""TT•Samu24bs""","""TARA""",11,1000,3,3,false,2025-07-13 15:21:58.011888
"""20250712T190858.000Z""","""unknown""","""unknown""","""hotZone""","""ranked""","""defeat""",70,"""#GQJRYV0JQ""","""TT•Samu24bs""","""STU""",11,1100,3,3,false,2025-07-13 15:21:58.011891
"""20250712T190328.000Z""","""unknown""","""unknown""","""hotZone""","""ranked""","""victory""",102,"""#GQJRYV0JQ""","""TT•Samu24bs""","""PIPER""",11,1000,3,3,false,2025-07-13 15:21:58.011895


In [26]:
battlelog_df["player_tag"].unique()

player_tag
str
"""#2GLQLRPJ2"""
"""#8C0PLGQRV"""
"""#RVUVR28LR"""
"""#VLGGJ2JG"""
"""#9JGLU0V8L"""
…
"""#2L8U99UGJ"""
"""#LUCUGYQLP"""
"""#VVPRGQ8J"""
